In [2]:
import os
import csv
import pandas as pd
from collections import Counter

In [3]:
home=%pwd
csv_path = home + '/llama_synth_level_one_output.csv'

In [4]:

# with open(csv_path, newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

In [5]:
df = pd.read_csv(csv_path)

In [6]:
df.shape

(1368, 7)

In [7]:
df.head(10)

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,Full_struct_corr
0,cube,1,0,0,1.0,NaN,0
1,tower,1,1,1,1.0,NaN,1
2,tower,1,1,1,1.0,NaN,1
3,tower,1,1,1,1.0,NaN,1
4,tower,1,1,1,1.0,NaN,1
5,tower,1,1,1,1.0,NaN,1
6,tower,1,1,1,1.0,NaN,1
7,tower,1,1,1,1.0,NaN,1
8,diamond,1,0,0,0.0,NaN,0
9,diamond,1,0,0,NaN,NaN,0


In [8]:
Counter(df['Shape_corr'].tolist())

Counter({1: 1003, 0: 365})

In [9]:
Counter(df['Size_corr'].tolist())

Counter({1: 941, 0: 427})

In [10]:
Counter(df['Color'].tolist())

Counter({1: 1364, 0: 4})

In [11]:
Counter(df['Full_struct_corr'].tolist())

Counter({0: 844, 1: 524})

In [ ]:
#Counter(df['Loc_corr'].tolist())

In [12]:
#which samples didn't get color right? Only when the moves were botched
df.loc[df['Color'] == 0]

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,Full_struct_corr
714,rectangle_botched,0,0,0,0.0,0.0,0
1050,rectangle_botched,0,0,0,0.0,0.0,0
1219,rectangle_botched,0,0,0,0.0,0.0,0
1331,rectangle_botched,0,0,0,0.0,0.0,0


In [13]:
#what the breakdown of correct and incorrect shapes?
#Correct:
Counter(df.loc[df['Shape_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 504,
         'row': 168,
         'diagonal': 132,
         'square': 128,
         'rectangle': 57,
         'cube': 14})

In [14]:
#Incorrect:
Counter(df.loc[df['Shape_corr'] == 0, 'Shape'].tolist())

Counter({'diamond': 144,
         'square': 88,
         'rectangle': 83,
         'diagonal': 36,
         'cube': 10,
         'rectangle_botched': 4})

In [15]:
#what the breakdown of correct and incorrect sizes?
#Correct:
Counter(df.loc[df['Size_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 504,
         'row': 168,
         'diagonal': 126,
         'square': 124,
         'cube': 12,
         'rectangle': 7})

In [16]:
#what the breakdown of correct and incorrect locations?
#Correct:
Counter(df.loc[df['Loc_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 212,
         'square': 62,
         'row': 43,
         'rectangle': 18,
         'cube': 12,
         'diagonal': 3})

In [25]:
Counter(df[(df['Size_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 504,
         'row': 168,
         'diagonal': 126,
         'square': 124,
         'cube': 12,
         'rectangle': 7})

In [24]:
Counter(df[(df['Loc_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 378,
         'row': 126,
         'diagonal': 102,
         'square': 88,
         'rectangle': 44,
         'cube': 8})

In [21]:
Counter(df[(df['Loc_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 212,
         'row': 43,
         'square': 23,
         'rectangle': 9,
         'cube': 3,
         'diagonal': 3})